In [1]:
from watson_developer_cloud import PersonalityInsightsV3, ToneAnalyzerV3, DocumentConversionV1
from pandas import *;from numpy import *;from io import open;from copy import copy
from sklearn.metrics.pairwise import pairwise_distances as distance
import re, nltk, os
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
nltk.download('stopwords')
from gensim import corpora, models, similarities
options.mode.chained_assignment = None
from ipywidgets import widgets
from IPython.display import display
from IPython.display import HTML

#from facebook import GraphAPI

#graph = GraphAPI(access_token="EAAEDpq4wxOkBAEhtTKnMotrJrreyXSK0COQA5qxzpEZCKnZAaHGpVIZBRlW6z4j1Z\
 #                A8A4B4nsdRd9MEheDF2sZCr6ujmhqvRMfWYKFppXRZAS5L0GLaRMnVFuUGk1fZBuhNcTEM8ZBeTpheh1xHmfz2w7r4Jv47beqnvs6GBAvPAsAZDZD", version="2.7")

convert = DocumentConversionV1(version='2017-09-10',username="01b7aea2-5b07-4c19-a733-012fde967ff7",
                                password="CAvAvuzYdAa8")
insights = PersonalityInsightsV3(version='2017-09-10',username="b7ecd1a7-1657-4dd9-b8d8-e6a8056fb37b",
                                 password="uPfXa6BGHSZx")
tone = ToneAnalyzerV3(version='2017-09-10',username="fd5b5c33-8082-4c7a-a781-7a792d632c25",
                      password="TzRHrhFOtyug")

[nltk_data] Downloading package stopwords to /home/daniel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Using TensorFlow backend.
Slow version of gensim.models.doc2vec is being used


In [26]:
def upload(filename):
    out = str('')
    if '.txt' in filename.lower():
        try:
            with open(filename, 'r') as f:
                for line in f.readlines():
                    out = out.replace('\n'," ")+line
        except:
            with open(filename, 'rb') as f:
                for line in f.readlines():
                    out = out.replace('\n'," ")+str(line).replace("\\r","").replace("\\n","").replace('b"',"")
    else:
        with open(filename,'rb') as document:
            config = {'conversion_target': DocumentConversionV1.NORMALIZED_TEXT}
            out = str(convert.convert_document(
                document=document, config=config).content)
            out = out.replace('\\n\\n',' ').replace('\\n',' ').replace('\\',' ')

    return out

def profile(text):
    
    #PERSONALITY
    character = insights.profile(text)
    
    df = []
    for k in ['needs', 'values']: df.append(DataFrame(character[k]))
    
    for j in range(5): df.append(DataFrame(character['personality'][j]['children']))
        
    data = df[0]
    for d in df[1:]: data = concat([data,d])
    
    del data['trait_id']
    data.columns = ['Type','Characteristic','Score']
    
    rows = data['Characteristic']+" ("+data['Type']+")"
    
    sideways = concat([rows, data["Score"]],1).T
    sideways.columns = rows
    sideways = DataFrame(sideways.iloc[1]).T
    
    #TONE
    tones = tone.tone(text, sentences = None)
    
    tones_all = [DataFrame(tones['document_tone']['tone_categories'][x]['tones']) for x in range(2)]
    tone_df = concat([tones_all[0],tones_all[1]])
    
    df = tone_df.T; df.columns = tone_df['tone_name']+" (Tone)"
    df = DataFrame(df.iloc[0]).T
    df.index = ['Score']
    df = df[["Analytical (Tone)","Confident (Tone)","Tentative (Tone)"]]
    y = []
    for x in text.split(" "):
        if x != '':y.append(x)
    df['Words'] = len(y)
    
    return concat([sideways, df],1), text

def events(*args):
    if type(args[0]) is list: 
        total_events = args[0][0]
        for j in args[0][1:]: total_events = concat([total_events,j])
    else: 
        total_events = args[0]
        for j in args[1:]: total_events = concat([total_events,j])
    return total_events

def closest(person, events):
    ds = distance(events, person, n_jobs = -1, metric = 'l2')
    return ds

def rank(me, data):    
    total = []
    for x in data.keys():
        total.append(data[x])
    concatted = events(total)
    concatted.index = data.keys()
    length_other = concatted.pop("Words")
    copy_me = copy(me)
    length_me = copy_me.pop("Words")

    scaling = matrix(length_other/length_other.mean()).T
    output = DataFrame(closest(copy_me, concatted))
    output.index = data.keys()

    output.columns = ["Distance"]

    output["Scaled"] = multiply(matrix(output["Distance"]).T,scaling)
    output["Scaled"] = output["Scaled"]/output["Scaled"].mean()
    output["Scaled"] = output["Scaled"]/output["Distance"].median()
    output["Result"] = output["Distance"]+output["Scaled"]

    output["Less than"] = 0
    output["Maximum"] = 0
    try:
        extra = copy_me.T["Score"].sort_values(ascending = False)
    except:
        extra = copy_me.T[0].sort_values(ascending = False)
    best = extra[extra>=0.8]

    for loc in range(len(concatted)):
        distances = (copy_me.iloc[0] - concatted.iloc[loc])**2
        total = sum(distances <= distances.mean()-1.96*distances.std()/(len(distances)**(0.5)))
        output["Less than"].iloc[loc] = total
        output["Maximum"].iloc[loc] = sum((concatted.iloc[loc][best.index])>=0.8)/len(best)

    output["Maximum"] = output["Maximum"]/output["Maximum"].mean()
    output["Max result"] = output["Maximum"].sum()-output["Maximum"]
    output["Max result"] = output["Max result"]/output["Max result"].mean()

    output["Less than result"] = output["Less than"]/output["Less than"].mean()
    output["Less than result"] = output["Less than"].sum()-output["Less than"]
    output["Less than result"] = output["Less than result"]/output["Less than result"].mean()

    output["Final"] = output["Distance"]*output["Result"]*output["Max result"]*output["Less than result"]

    output.index = data.keys()
    
    return output["Final"].sort_values(), concatted


def process(args):
    lda = []; files = []; profiles = []
    for file in args:
        obj = upload(file)
        files.append(obj)
        
        res = re.split(r"[^a-zA-Z]",obj)
        end = []
        for x in res:
            if x != '' and len(x) >= 2 and len(x) <= 12: end.append(x.lower().lstrip().rstrip())
        good = [word for word in end if word not in stopwords.words('english')]

        p_stemmer = PorterStemmer()
        result = [p_stemmer.stem(i) for i in good]
        lda.append(result)
        
        profiler = profile(obj)
        profiles.append(profiler)
        
    data = {x:y for x,y in zip(args, profiles)}
    ldas = {x:y for x,y in zip(args, lda)}
    
    return data, ldas

def folder(where):
    files = []
    for file in os.listdir(where):
        if file.endswith(".ipynb") or file.endswith(".ipynb_checkpoints"): pass
        else:
            files.append(os.path.join("", file))
    return files

def getme(data, d):
    out = data[d]
    del data[d]
    return out

def similar(notgood, me, lsi):
    dfs = [lsi[x] for x in lsi.keys()]
    dictionary = corpora.Dictionary(dfs)
    corpus = [dictionary.doc2bow(text) for text in dfs]
    modeller = models.LsiModel(corpus, id2word=dictionary, num_topics=2)
    vec_bow = dictionary.doc2bow(me[1].lower().split())
    vec_lsi = modeller[vec_bow]
    index = similarities.MatrixSimilarity(modeller[corpus])
    sims = index[vec_lsi]
    sims = sorted(enumerate(sims), key=lambda item: -item[1])

    all = [(list(lsi.keys())[x],y) for x,y in sims]
    b = DataFrame(all)
    b.index = b[0]
    b = b[1]
    b = DataFrame(b)
    b.columns = ["Similarity"]
    return b[b.index!=notgood]["Similarity"]

def recommend(me,lsi,data, top = 10):
    sim = similar('me.txt',me,lsi)
    ranks, concatted = rank(me[0], {x:data[x][0] for x in data.keys()})
    
    df = merge(DataFrame(sim).reset_index(), DataFrame(ranks).reset_index(), left_on = 0, 
               right_on = "index")

    df = df[["index", "Similarity", "Final"]]

    reps = ((df["Similarity"]/df["Similarity"].mean())*df["Similarity"].std())
    df["Score"] = df["Final"]/df["Final"].mean()-(reps-reps.max())

    df["Norm"] = df["Score"]/df["Score"].sum()
    scale = ((df["Norm"].max() - df["Norm"])*100)
    df["%Rank"] = (scale*(100-scale.max())/10).round(2)
    
    df.index = df["index"]    
    
    output = DataFrame(df["%Rank"].sort_values(ascending = False))
    output = output.reset_index()
    output.columns = ["File","Match"]
    
    source = []
    for u in output["File"]:
        try:
            source.append(get_index()[u.split('.txt')[0]]['site'])
        except:
            source.append('Subject')
            
    output["Source"] = Series(source).apply(lambda x: '<a href="{}" target="_blank">link</a>'.format(x) 
                                            if x!='Subject' else 'Subject')
    
    output['File'] = output['File'].apply(lambda x: x.replace('.txt',''))
    can_change = copy(output)
    
    if top == 'all':
        return output.style.bar(subset = ["Match"], color = 'grey'), can_change
    else:
        return output.iloc[0:top].style.bar(subset = ["Match"], color = 'grey'), can_change
    
def to_update(where):
    total = folder(where)
    seen = []
    where = where+'/data'
    for file in os.listdir(where):
        if file.endswith(".ipynb") or file.endswith(".ipynb_checkpoints"): pass
        else:
            seen.append(os.path.join("", file))
    seen = [x for x in seen if '.txt' in x]
    return [x for x in total if x not in seen and '.txt' in x and x != 'index.txt' and 'profile' not in x]

def save_as(file): file.to_csv(file.split('.')[0]+'.csv')
    
def get_index():
    splitted = [x.split('(') for x in open("index.txt").read().split('\n')]
    splitted = [[x[0].rstrip(), x[1].split(')')[0]] for x in splitted]
    p = [u+[u[0].split('_')[0]]+[u[0].split('_')[1]] for u in splitted]
    index = {x[0]:{'site':x[1], 'subject':x[2].title(), 'title':x[3].title()} for x in p}
    return index

def process_update(where):
    what = to_update(where)
    for file in what:
        result = process([file])
        result[0][file][0].to_csv('data/'+file.split('.txt')[0]+'.csv', index = False)
        DataFrame([result[0][file][1]]).to_csv('data/'+file)

        DataFrame(result[1]).to_csv('words/'+file.split('.txt')[0]+'.csv', index = False)
        
        
def read_in(where):
    out = folder(where+'/data')
    wds = folder(where+'/words')

    texts = sorted([x for x in out if '.txt' in x])
    csvs = sorted([x for x in out if '.csv' in x])
    text = {}
    ldas = {}
    for c,y in zip(csvs,texts):
        text[y] = [read_csv(where+'/data/'+c)]
        text[y].append(str(open(y).read()).replace('\n',''))
        ldas[y] = (read_csv(where+'/words/'+c))[y].tolist()
        
    return text, ldas

def explore(where):
    process_update(where)
    return read_in(where)

def document(where):
    upload()
    
def subjects():
    subjects = {}
    for i in get_index().keys():
        try:
            subjects[get_index()[i]['subject']].append(i+'.txt')
        except:
            subjects[get_index()[i]['subject'].title()] = [i+'.txt']
    out = subjects
    subjs = {}
    for i in out.keys():
        combined = ''
        for u in out[i]:
            combined += data[u][1]
        subjs[i] = combined

    for p in subjs.keys():
        DataFrame([subjs[p]]).to_csv(p+".txt", index = False)
        
def reset_profile():
    file = open('me.txt', mode = 'w')
    try:
        os.remove("data/me.csv");
    except:
        pass;
    try:
        os.remove("data/me.txt"); 
    except:
        pass;
    try:
        os.remove("words/me.csv")
    except:
        pass;
    file.write('"1. What is you favourite movie, TV show or song and why?"\n\n\n')
    file.write('"2. If you had to choose a country to stay in forever, which would it be and why?"\n\n\n')
    file.write('"3. Given you only have 1 hour before the world ends, what will you do and why?"\n\n\n')
    file.write('"4. You have 2 subjects to choose from. Maths OR English. Which and why?"\n\n\n')
    
def read_profile():
    file = open('me.txt', mode = 'r')
    new = open('profile.txt', mode = 'w')
    for line in file:
        new.write(line)
    file = open('me.txt', mode = 'w')
    file.write(str([x for x in open('profile.txt').readlines()]))
    
def recommend_subjects(me,lsi,data):
    ranks,df = recommend(me,lsi,data, top = 'all')
    df = df[df["Source"] == "Subject"][["File","Match"]]
    return df.style.bar(subset = ["Match"], color = 'grey')

def recommend_events(me,lsi,data,top=10):
    ranks,df = recommend(me,lsi,data, top = top)
    if top == 'all': df = df[df["Source"] != "Subject"]
    else:  df = df[df["Source"] != "Subject"].iloc[0:top]
    return df.style.bar(subset = ["Match"], color = 'grey')

def personality_top(me, top = 2):
    t = me[0].T[0]
    needs = t[[x for x in list(t.index) if "needs" in x]].sort_values(ascending = False)[0:top]
    values = t[[x for x in list(t.index) if "values" in x]].sort_values(ascending = False)[0:top]
    personality = t[[x for x in list(t.index) if "personality" in x]].sort_values(ascending = False)[0:top]
    Tone = t[[x for x in list(t.index) if "Tone" in x]].sort_values(ascending = False)[0:top]
    data = concat([needs,values,personality,Tone])
    return DataFrame(data).style.bar(subset = [0], color = 'grey')

def personality_bot(me, bot = 2):
    t = me[0].T[0]
    needs = t[[x for x in list(t.index) if "needs" in x]].sort_values(ascending = True)[0:bot]
    values = t[[x for x in list(t.index) if "values" in x]].sort_values(ascending = True)[0:bot]
    personality = t[[x for x in list(t.index) if "personality" in x]].sort_values(ascending = True)[0:bot]
    Tone = t[[x for x in list(t.index) if "Tone" in x]].sort_values(ascending = True)[0:bot]
    data = concat([needs,values,personality,Tone])
    return DataFrame(data).style.bar(subset = [0], color = 'grey')

In [22]:
reset_profile()

In [27]:
data, lsi = explore('/home/daniel/Desktop/maker')
me = getme(data, 'me.txt')

In [28]:
personality_top(me)

,0
Love (needs),0.566
Closeness (needs),0.548655
Conservation (values),0.431333
Hedonism (values),0.323475
Orderliness (personality),0.997833
Dutifulness (personality),0.96546
Analytical (Tone),0.269672
Tentative (Tone),0.013155


In [29]:
personality_bot(me)

,0
Practicality (needs),0.0509169
Curiosity (needs),0.0820283
Openness to change (values),0.26378
Self-enhancement (values),0.288426
Excitement-seeking (personality),0.0839211
Assertiveness (personality),0.198292
Confident (Tone),0
Tentative (Tone),0.013155


In [30]:
ranks,df = recommend(me,lsi,data, top = 10)
ranks

,File,Match,Source
0,neuroscience_neuralaunchesonlinetalkseriesformentalhealthweek,14.18,link
1,pharmacy_explaingdrugfromresearchtomarket,14.06,link
2,CSE_High School Information Day 2017,13.59,link
3,CSE_a robotics adventure in Poland,13.31,link
4,medicine_mostivfbabiescomefromfrozenembryo,13.03,link
5,CSE_when arts lead to science,12.72,link
6,psychology_snoosphereatthebiganxietyfestival,11.68,link
7,engineering_solar race challenge,11.42,link
8,Pharmacy,11.38,Subject
9,CSE_CSESOC events,11.13,link


In [31]:
recommend_subjects(me,lsi,data)

,File,Match
8,Pharmacy,11.38
11,Mathematics,10.32
15,Medicine,9.72
16,Space,9.7
20,Staff,9.21
22,Built Environment,9.03
25,Engineering,8.35
29,Science,6.9
36,Environment,6.38
37,Neuroscience,6


In [32]:
recommend_events(me,lsi,data, top = 5)

,File,Match,Source
0,neuroscience_neuralaunchesonlinetalkseriesformentalhealthweek,14.18,link
1,pharmacy_explaingdrugfromresearchtomarket,14.06,link
2,CSE_High School Information Day 2017,13.59,link
3,CSE_a robotics adventure in Poland,13.31,link
4,medicine_mostivfbabiescomefromfrozenembryo,13.03,link
